In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import json
from sqlalchemy import create_engine, text
# Set display.max_columns to None to show all columns
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')


In [431]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Your SQL query
query = """
    your big query query
"""

### Calling Big Query table 

In [ ]:
client = bigquery.Client.from_service_account_json(**path to your big query service account key for authorization**)


# Execute the query and fetch results
query_job = client.query(query)  # API request

df = query_job.to_dataframe(create_bqstorage_client=True)


# # Retrieve query results
# results = query_job.result()  # Waits for job to complete

# for row in results:
#     for key, value in row.items():
#         print(f"{key}: {value}")

In [434]:
df.shape

(8805, 30)

## New Flattening Code 

#### Event Params Column Flattening

In [436]:
# Function to flatten the event_params
def flatten_event_params(row):
    flattened = {}
    for item in row:
        key = item['key']
        value_dict = item['value']
        # Get the non-null value from the value dict
        value = next((v for v in value_dict.values() if v is not None), None)
        flattened[f"event_params_{key}"] = value
    return flattened

# Apply the function and expand the results into separate columns
flattened_df = pd.DataFrame(df['event_params'].apply(flatten_event_params).tolist())

In [437]:
# Drop the original event_params column
df = df.drop(columns=['event_params'])

# Concatenate with the original DataFrame if needed
df = pd.concat([df, flattened_df], axis=1)

In [439]:
df.shape

(8805, 71)

#### privacy_info column flattening

In [441]:
# Function to flatten the privacy_info column
def flatten_privacy_info(row):
    flattened = {}
    for key, value in row.items():
        flattened[f"privacy_info_{key}"] = value
    return flattened

# Apply the function and expand the results into separate columns
flattened_privacy_info_df = pd.DataFrame(df['privacy_info'].apply(flatten_privacy_info).tolist())

# Drop the original privacy_info column
df = df.drop(columns=['privacy_info'])

# Concatenate the flattened columns with the original DataFrame
df = pd.concat([df, flattened_privacy_info_df], axis=1)

In [443]:
df.shape

(8805, 73)

#### user_properties column flattening

In [445]:
# Function to flatten the user_properties column
def flatten_user_properties(row):
    flattened = {}
    for item in row:
        key = item['key']
        value_dict = item['value']
        
        # Pick the correct non-null value based on your rules
        value = (
            value_dict['string_value'] or 
            value_dict['int_value'] or 
            value_dict['float_value'] or 
            value_dict['double_value']
        )
        
        flattened[f"user_properties_{key}"] = value
    return flattened

# Apply the function and expand the results into separate columns
flattened_user_properties_df = pd.DataFrame(df['user_properties'].apply(flatten_user_properties).tolist())

flattened_user_properties_df = flattened_user_properties_df.drop(columns=['user_properties_dob', 'user_properties_doj',
                                  'user_properties_age', 'user_properties_gc_uf_loyality', 'user_properties_name'])

# # Drop the original user_properties column
df = df.drop(columns=['user_properties'])

# # Concatenate the flattened columns with the original DataFrame
df = pd.concat([df, flattened_user_properties_df], axis=1)


In [446]:
df.shape

(8805, 82)

#### device column flattening

In [448]:
# Function to flatten the device column
def flatten_device_info(row):
    flattened = {}
    for key, value in row.items():
        flattened[f"device_{key}"] = value
    return flattened

# Apply the function and expand the results into separate columns
flattened_device_df = pd.DataFrame(df['device'].apply(flatten_device_info).tolist())

# Drop the original device column
df = df.drop(columns=['device'])

# Concatenate the flattened columns with the original DataFrame
df = pd.concat([df, flattened_device_df], axis=1)

In [449]:
df.shape

(8805, 96)

#### geo col flattening

In [451]:
# Function to flatten the geo column
def flatten_geo_info(row):
    flattened = {}
    for key, value in row.items():
        flattened[f"geo_{key}"] = value
    return flattened

# Apply the function and expand the results into separate columns
flattened_geo_df = pd.DataFrame(df['geo'].apply(flatten_geo_info).tolist())

# Drop the original geo column
df = df.drop(columns=['geo'])

# Concatenate the flattened columns with the original DataFrame
df = pd.concat([df, flattened_geo_df], axis=1)

In [452]:
df.shape

(8805, 101)

#### app_info col

In [453]:
# Function to flatten the app_info column
def flatten_app_info(row):
    flattened = {}
    for key, value in row.items():
        flattened[f"app_info_{key}"] = value
    return flattened

# Apply the function and expand the results into separate columns
flattened_app_info_df = pd.DataFrame(df['app_info'].apply(flatten_app_info).tolist())

# Drop the original app_info column
df = df.drop(columns=['app_info'])

# Concatenate the flattened columns with the original DataFrame
df = pd.concat([df, flattened_app_info_df], axis=1)

In [454]:
df.shape

(8805, 105)

#### traffic_source col 

In [455]:
# Function to flatten the traffic_source column
def flatten_traffic_source(row):
    flattened = {}
    for key, value in row.items():
        flattened[f"traffic_source_{key}"] = value
    return flattened

# Apply the function and expand the results into separate columns
flattened_traffic_source_df = pd.DataFrame(df['traffic_source'].apply(flatten_traffic_source).tolist())

# Drop the original traffic_source column
df = df.drop(columns=['traffic_source'])

# Concatenate the flattened columns with the original DataFrame
df = pd.concat([df, flattened_traffic_source_df], axis=1)

In [456]:
df.shape

(8805, 107)

#### items col

In [457]:
df['items'][0]

array([], dtype=object)

In [458]:
# Replace arrays that are empty with None (equivalent to NULL)
df['items'] = df['items'].apply(lambda x: None if isinstance(x, np.ndarray) and x.size == 0 else x)

df.shape


(8805, 107)

####  collected_traffic_source flattening

In [459]:
def flatten_collected_traffic_source(row):
    # Initialize an empty dictionary to hold the flattened data
    flattened = {}

    # Ensure the row is not None
    if row:
        # Loop through each key-value pair in the dictionary
        for key, value in row.items():
            # Only add the key-value pair if the value is not None
            flattened[f"collected_traffic_source_{key}"] = value

    # Ensure that all expected columns are present, even if they are None
    expected_columns = [
        'manual_campaign_id', 'manual_campaign_name', 'manual_source', 'manual_medium', 
        'manual_term', 'manual_content', 'manual_source_platform', 'manual_creative_format',
        'manual_marketing_tactic', 'gclid', 'dclid', 'srsltid'
    ]

    for col in expected_columns:
        if f"collected_traffic_source_{col}" not in flattened:
            flattened[f"collected_traffic_source_{col}"] = None

    return flattened

# Apply the function and expand the results into separate columns
flattened_collected_traffic_source_df = pd.DataFrame(df['collected_traffic_source'].apply(flatten_collected_traffic_source).tolist())


# Drop the original traffic_source column
df = df.drop(columns=['collected_traffic_source'])

# Concatenate the flattened columns with the original DataFrame
df = pd.concat([df, flattened_collected_traffic_source_df], axis=1)

#### session_traffic_source_last_click col flattening

In [462]:
def flatten_session_traffic_source_last_click(row):
    # Initialize an empty dictionary to hold the flattened data
    flattened = {}

    # Handle the 'manual_campaign' nested dictionary separately
    if 'manual_campaign' in row and row['manual_campaign'] is not None:
        manual_campaign = row['manual_campaign']
        for key, value in manual_campaign.items():
            flattened[f"session_traffic_source_last_click_manual_campaign_{key}"] = value
    else:
        # Ensure that all expected keys from 'manual_campaign' are added with None if not available
        manual_campaign_keys = [
            'campaign_id', 'campaign_name', 'source', 'medium', 'term', 'content', 
            'source_platform', 'creative_format', 'marketing_tactic'
        ]
        for key in manual_campaign_keys:
            flattened[f"session_traffic_source_last_click_manual_campaign_{key}"] = '(not set)'

    # Handle the rest of the columns in the row
    expected_columns = [
        'google_ads_campaign', 'cross_channel_campaign', 'sa360_campaign',
        'cm360_campaign', 'dv360_campaign'
    ]
    
    for col in expected_columns:
        flattened[f"session_traffic_source_last_click_{col}"] = row.get(col, None)

    return flattened

# Apply the function and expand the results into separate columns
flattened_session_traffic_source_last_click_df = pd.DataFrame(df['session_traffic_source_last_click'].apply(flatten_session_traffic_source_last_click).tolist())

# Drop the original traffic_source column
df = df.drop(columns=['session_traffic_source_last_click'])

# Concatenate the flattened columns with the original DataFrame
df = pd.concat([df, flattened_session_traffic_source_last_click_df], axis=1)

In [464]:
df.shape

(8805, 131)

### Data Transformation 

#### Date Cols Transformation

In [ ]:
# List of date columns (based on your DataFrame's column names)
date_columns = []

# Convert date columns to date format
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce').dt.date


#### ID Cols Transformation 

In [ ]:
# List of ID columns to convert to integer
id_columns = []

# Convert ID columns to integer format
for col in id_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce', downcast='integer')


#### Time cols Transformation  code below 👇🏼

In [481]:
# Convert columns containing dicts to JSON format (if they exist)
for column in df.columns:
    if df[column].apply(lambda x: isinstance(x, dict)).any():
        df[column] = df[column].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)


### Loading data from final_df to the staging table

In [ ]:
engine = create_engine(podtgresdb host port username password)

In [ ]:
# Replace 'your_schema' and 'staging_table' with your actual schema and table names
df.to_sql(table_name, engine, schema, if_exists='replace', index=False)


203

In [487]:
engine.dispose()